# Is connect improving success prob in student with weaker technical background?

### Method: 
* Extract programming skills on each students; separate general and connect, then do a logistic regression on both to see p value (smaller p value means stronger reliance on programming skills). ideally in connect the p value supposed to be bigger (less sig.);
* Do a viz? x-axis: programming skill level; y-axis: graducation

In [16]:
# read data from Laurel (ud208 is from Hardy, but imported into Laurel before this)
import psycopg2
import pandas as pd
import seaborn as sns
import pymc3 as pm
import numpy as np
import random
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
%matplotlib inline

In [2]:
conn_laurel = psycopg2.connect("dbname='analytics' user='analytics' host='analytics.cv90snkxh2gd.us-west-2.rds.amazonaws.com' password='!TgP$Ol9Z&6QhKW0tmn9mOW5rYT2J8'")
#conn_hardy = psycopg2.connect("dbname='analytics' user='u_yidong' host='udacity-segment.c2zpsqalam7o.us-west-2.redshift.amazonaws.com' port='5439' password='Lyd580809?!'")

sql_apps = "SELECT distinct a.id as application_id,a.cohort_id,a.applicant_id,a.nd_key,a.application_type\
      ,coalesce(a.applicant_country,'Unknown') as applicant_country\
      ,coalesce(a.applicant_geo,'Unknown') as applicant_geo\
      ,case when reg.state = 'GRADUATED' or grad.graduated_at is not null then 'graduated'\
            when b.user_id is not null then 'term completed'\
            when a.first_charge_created_at is not null and a.accepted_at is not null then 'paid'\
            when a.accepted_at is not null then 'accepted'\
            when a.rejected_at is not null then 'rejected'\
            when a.submitted_at is not null then 'submitted'\
       else 'applied' end as application_status\
      ,current_date - a.applicant_created_at::date as user_age\
      ,a.cohort_notify_at::date - a.cohort_open_at::date as cohort_open_to_notify\
      ,a.cohort_close_at::date - a.cohort_open_at::date as cohort_open_to_close\
      ,date_part('month',a.cohort_open_at) as cohort_open_month\
      ,date_part('month',a.application_created_at) as application_month\
      ,a.application_created_at::date - a.cohort_open_at::date as apply_before_start\
      ,a.submitted_at::date - a.application_created_at::date as apply_to_submit\
      ,coalesce(a.accepted_at,a.rejected_at)::date - a.submitted_at::date as submit_to_decision\
      ,a.cohort_notify_at::date - a.submitted_at::date as submit_to_notify\
      ,greatest(coalesce(c.first_charge_attempt::date\
      ,a.first_charge_created_at::date) - a.cohort_notify_at::date,0) as notify_to_pay\
      ,coalesce(a.price,0)::int as price\
       FROM admissions.applications as a\
       LEFT JOIN enrollments.term_completions as b ON a.applicant_id = b.user_id and a.nd_key = b.nd_key\
       LEFT JOIN (select distinct account_key,nd_key,cohort_id,min(charge_created_at) as first_charge_attempt\
                  from payments.charges\
                  group by account_key,nd_key,cohort_id) as c\
             ON a.applicant_id = c.account_key and substring(a.nd_key,0,6) = substring(c.nd_key,0,6) \
            and (case when a.application_type = 'term' then a.cohort_id::int else 0 end) = coalesce(c.cohort_id,0)\
       LEFT JOIN enrollments.nanodegree_graduations_segment AS grad\
         ON a.applicant_id = grad.user_id AND substring(a.nd_key,0,6) = substring(grad.nanodegree,0,6)\
       LEFT JOIN enrollments.nanodegree_enrollments_registrar AS reg\
         ON a.applicant_id = reg.user_id and substring(a.nd_key,0,6) = substring(reg.course_key,0,6)\
       WHERE accepted_at is not null and first_charge_created_at is not null\
         AND email not like '%@udacity%';"
sql_questions = "select distinct application_id,cohort_id,applicant_id,nd_key\
                        ,question_id,question_prompt,response,question_prompt||' A:'||response as q_r\
                from admissions.question_responses\
                where application_type != 'scholarship' and response is not null\
                  and question_id in ('question_id','48e7b492-62b4-4d99-b596-80d68f2966ae','cad2565e-8176-44b5-9080-14cfc67ea9b4','fba3666b-db04-46e9-8f3d-2a303f13e0a5','26330483-09a3-444a-b534-e9cd7021ec1c','42b96348-56c4-4e3b-b0f9-8d3911969e48','516737fe-4567-48d2-8645-08d335e21b0c','0c22dbaa-bf1d-4c67-be34-a61014591d6f','2ad03aaa-1b35-4381-9c43-907b1b4eba67','6afe0061-746b-4bd7-807c-393fe5c7599d','779c3b6c-3648-423b-8d3f-8a4f36f23e2a','1346c396-a687-4082-a10e-d3556ac50b4b','1cfe0061-746b-4bd7-807c-393fe5c7599d','ed06fc88-6a59-40a8-b7ea-ece90d83c333','fa332682-b62e-4791-b95f-c528b532a754','e42e1049-e459-4602-ae4b-cb78098dcf89','01f28833-152f-438d-a831-51e33583bde5','7c55f3cc-1c0b-4d95-97f9-cc3409e3145f','a633a338-9726-472a-a0ff-0d8695b30638','f1a93ec7-e2cd-46b6-ba28-2f36b30539a9','6967091c-09c6-4455-9f1e-d0de318bacc5','011ef02c-abea-476d-afdc-d1aa8a456b94','0f617540-5680-480c-aaf3-0663c17c3c7d','1dfe0061-746b-4bd7-807c-393fe5c7599d','20f21bd0-eb3e-4898-ae1f-eb994f9ce823','45781b84-093b-41c7-b5a4-c521f343bf21','47d35057-b1fb-4106-8218-7e551497420e','491b500b-26a2-425b-b142-00773f1d1938','5cf34496-6bc3-4c8d-9c6a-7c037f48669f','70ffc637-2162-484a-b863-3a1a1469f106','7a09e78a-8000-459e-9d79-aa21db135138','823a8248-b7d7-4dbe-a619-2969ca453f1e','8afe0061-746b-4bd7-807c-393fe5c7599d','8ea6732d-0ef9-489d-bf11-20673183a0c2','9481dc6b-c4e9-4207-a30c-65c3ac302bf5','995331ea-092b-4b71-8122-a0c65214c765','9d5c6390-a690-4d6d-8aaa-c60a772b4550','a26ac91f-c263-49f1-8171-9821bc4d2da5','a71ff6e3-1b33-4e11-94f1-5b94d786b5ab','a7b7f95b-1924-4189-840e-b5c407545908','ae155453-72b2-4247-8da5-dced5270f29b','bdc024d8-7712-48ff-b86e-292be1df0d9f','d3b0e6c7-0601-4651-851f-7ef4bea1aa6d','d8f9089c-c3a5-4d77-9c37-ae248e134e42','d9af077d-b729-4971-84b8-f86f390789c1','da04a859-199c-42cf-ab5d-6132bc529161','dc898f2e-1810-4d93-9db0-f381778d45e1','f254514a-26f4-4d29-83af-711c27d4862d','f34c024a-b446-464e-add0-558c9eb8a3c8','fa537969-017a-4816-a317-b5883305c723')"

df_apps = pd.read_sql(sql_apps,conn_laurel)
df_questions = pd.read_sql(sql_questions,conn_laurel)

df = pd.merge(df_apps,df_questions,on=['application_id','cohort_id','applicant_id','nd_key'],how='inner')

In [3]:
# programming languages
data = df
for i in (['python','java','cplus','probability','statistics','linear algebra','computer science','machine learning']):
    if i == 'cplus':
        j = 'c\+\+' 
    else:
        j = i
    d = df_questions.copy()
    d[i] = 0
    d[i] = np.where(d['response'].str.lower().str.contains(j),1,d[i])
    d[i] = np.where(d['question_prompt'].str.lower().str.contains(j)&d['response'].str.match('.*[1-9].*'),1,d[i])
    o_i = d.groupby('application_id').agg({i:'max'}).reset_index()
    data = pd.merge(data,o_i,on=['application_id'],how='inner')

In [4]:
data.groupby(['nd_key','application_status'])['application_id'].count()

nd_key              application_status
nd001-connect       paid                     10
nd002-connect       paid                    200
nd004-connect-reno  graduated                30
                    paid                    170
nd009-connect       paid                    286
nd013               graduated                40
                    paid                  26792
                    term completed         7591
nd209               graduated                48
                    paid                  14230
                    term completed         1274
nd889               paid                  10848
                    term completed          270
Name: application_id, dtype: int64

In [5]:
data['status'] = data['application_status'].map({'graduated':1,'term completed':1,'paid':0})

In [6]:
data.groupby(['status'])['application_id'].count()

status
0    52536
1     9253
Name: application_id, dtype: int64

In [7]:
data.groupby(['status','application_type'])['application_id'].count()

status  application_type
0       connect               666
        term                51870
1       connect                30
        term                 9223
Name: application_id, dtype: int64

In [19]:
df_connect = data.loc[data['application_type']=='connect']
df_connect_grad = df_connect.loc[df_connect['status']==1]
df_connect_enrol = df_connect.loc[df_connect['status']==0]
df_general = data.loc[data['application_type']=='term']
df_general_grad = df_general.loc[df_general['status']==1]
df_general_enrol = df_general.loc[df_general['status']==0]

In [15]:
n_grad = df_connect_grad.shape[0]
n_grad

30

In [23]:
df_connect_enrol_2 = df_connect_enrol.iloc[random.sample(range(df_connect_enrol.shape[0]),n_grad),:]
df_general_grad_2 = df_general_grad.iloc[random.sample(range(df_general_grad.shape[0]),n_grad),:]
df_general_enrol_2 = df_general_enrol.iloc[random.sample(range(df_general_enrol.shape[0]),n_grad),:]

In [24]:
df = pd.concat([df_connect_grad,df_connect_enrol_2,df_general_grad_2,df_general_enrol_2],axis=0)

In [26]:
df.groupby(['status','application_type'])['application_id'].count()

status  application_type
0       connect             30
        term                30
1       connect             30
        term                30
Name: application_id, dtype: int64

In [27]:
df.columns

Index(['application_id', 'cohort_id', 'applicant_id', 'nd_key',
       'application_type', 'applicant_country', 'applicant_geo',
       'application_status', 'user_age', 'cohort_open_to_notify',
       'cohort_open_to_close', 'cohort_open_month', 'application_month',
       'apply_before_start', 'apply_to_submit', 'submit_to_decision',
       'submit_to_notify', 'notify_to_pay', 'price', 'question_id',
       'question_prompt', 'response', 'q_r', 'python', 'java', 'cplus',
       'probability', 'statistics', 'linear algebra', 'computer science',
       'machine learning', 'status'],
      dtype='object')

In [28]:
cols = ['python', 'java', 'cplus','probability', 'statistics', 'linear algebra', 'computer science','machine learning']

In [32]:
# Logistic Regression on Connect
logit = sm.Logit(df_connect['status'], df_connect[cols])
result = logit.fit()
print(result.summary())

         Current function value: 0.341716
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                 status   No. Observations:                  696
Model:                          Logit   Df Residuals:                      688
Method:                           MLE   Df Model:                            7
Date:                Sun, 12 Nov 2017   Pseudo R-squ.:                 -0.9232
Time:                        21:20:57   Log-Likelihood:                -237.83
converged:                      False   LL-Null:                       -123.67
                                        LLR p-value:                     1.000
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
python              -0.8760      0.240     -3.645      0.000      -1.347      -0.405
java                -0.5694      0.252     -2.2

/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [33]:
# Logistic Regression on General
logit = sm.Logit(df_general['status'], df_general[cols])
result = logit.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.451875
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 status   No. Observations:                61093
Model:                          Logit   Df Residuals:                    61085
Method:                           MLE   Df Model:                            7
Date:                Sun, 12 Nov 2017   Pseudo R-squ.:                -0.06478
Time:                        21:21:51   Log-Likelihood:                -27606.
converged:                       True   LL-Null:                       -25927.
                                        LLR p-value:                     1.000
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
python               0.9821      0.034     28.815      0.000       0.915       1.049
java       

## Two Logistic Regression Models show that programming skills are very significant influencing the graducation of general nanodegree; connect however at the other hand, the graducation does not reply as heavily as general nanodegree, on programming skills.